## 分子体系二次量子化哈密顿量的FCI方法求解
上面我们介绍了FCI方法的框架，但是上面写的并不适用于任意的二次量子化哈密顿量，因为上面的哈密顿量矩阵元计算是利用了Hubbard Model的一些性质的简化计算。由于Hubbard Model的双电子项只有对角项，其他都为零，其计算哈密顿量矩阵元较为简单。如果是任意一个二次量子化哈密顿量：
$$
\hat{H}=\sum_{p q} h_{p q} a_p^{\dagger} a_q+\sum_{p q r s} v_{p q}^{r s} a_p^{\dagger} a_q^{\dagger} a_r a_s
$$
大致的FCI计算思路还是一样的，先构建Hamiltonian矩阵然后对角化。不过哈密顿量矩阵元的计算有点不同。

我们还是先把一些重要的东西，比如单双电子积分算出来

In [1]:
from pyscf import gto, scf, ao2mo
import numpy as np
from functools import reduce

def geometry_Hn(n=2,dist = 1.0):
    geometry = []
    for i in range(n):
        geometry.append(("H",(0,0,i*dist)))
    return geometry

mol = gto.Mole()
mol.atom = geometry_Hn(n=8)
mol.basis = "sto-3g"
mol.build()

mf = scf.RHF(mol)
mf.run()

norb = mol.nao_nr()
n_spin_orb = 2 * norb
Sz = mol.spin
nelec = mol.nelectron
nelec_a = (nelec + Sz) //2
nelec_b = (nelec - Sz) //2
ecore = mol.energy_nuc()
mo_coeff = mf.mo_coeff

h1e_ao = mol.intor("int1e_nuc") + mol.intor("int1e_kin")
h1e = reduce(np.dot, (mo_coeff.T, h1e_ao, mo_coeff))

eri_ao = mol.intor("int2e")
eri = np.einsum(
    "ijkl, ip, js, kq, lr->psqr",
    eri_ao, mo_coeff.conj(), mo_coeff,
    mo_coeff.conj(), mo_coeff)

converged SCF energy = -4.17436981038916


上面得到的`h1e`和`eri`就是分子轨道(MO)基组下的单双电子积分
$$
\hat{H}=\sum_{p q} h_{p q} a_p^{\dagger} a_q+\sum_{p q r s} v_{p q}^{r s} a_p^{\dagger} a_q^{\dagger} a_r a_s
$$
`h1e`和`eri`分别对应了二次量子化哈密顿量中的$h_{p q}$和$v_{p q}^{r s}$

然后我们还是一样要确定希尔伯特空间中具体有哪些组态

In [2]:
from itertools import combinations
import math

def get_num_configuration(norb, nelec, Sz=0):
    nelec_a = (nelec + Sz) //2
    nelec_b = (nelec - Sz) //2
    na = math.comb(norb,nelec_a)
    nb = math.comb(norb,nelec_b)
    return na*nb

def get_all_configurations(norb,nelec,Sz=0):
    nelec_a = (nelec + Sz) //2
    nelec_b = (nelec - Sz) //2
    configurations = []
    for spin_up_occ in combinations(range(norb), nelec_a):
        for spin_dn_occ in combinations(range(norb), nelec_b):
            configuration = np.zeros(2*norb) # 定义一个空组态，其长度为2*norb，等于自旋轨道数。自旋轨道的排列顺序为： 1↑ 1↓ 2↑ 2↓ 3↑ 3↓ ...
            spin_up_occ = np.array(spin_up_occ)
            spin_dn_occ = np.array(spin_dn_occ)
            configuration[2*spin_up_occ] = 1        # 根据spin_up_occ把占据的spin up obritals设置为1
            configuration[2*spin_dn_occ + 1] = 1    # 根据spin_dn_occ把占据的spin down obritals设置为1
            configurations.append(configuration)
    configurations = np.array(configurations)
    return configurations

configurations = get_all_configurations(norb,nelec)

并且一样把组态(configuration)转化成十进制数(string)进行存储

In [3]:
def config_to_string(configurations:np.array):
    configurations = np.array(configurations,dtype = np.int64)
    n_configurations = configurations.shape[0] # 组态数
    n_spin_orbs = configurations.shape[1]   # 自旋轨道数
    strings = np.zeros(n_configurations, dtype = np.int64) # 先定义一个空的new_configurations的数组，里面用十进制数存组态
    for i in range(n_spin_orbs):
        strings += configurations[:,i]* 2**i # 每一位数值代表了2**i
    return strings

ci_strings = config_to_string(configurations)
n_strings = n_configurations = ci_strings.shape[0]

之后还要先做一点准备工作：

In [4]:
def cre_des_sign(p, q, string0):
    """
    这个函数的作用是，对于一个组态string0，它对于第p个轨道作用一个产生算符，第q个轨道作用一个湮灭算符，得到的新组态string1, 
    string0 和string1之间由交换反对称确定的符号是+1还是-1

    Args:
        p (int): 产生算符作用的位置
        q (int): 湮灭算符作用的位置
        string0 (int): 以十进制数保存的组套

    Returns:
        sign: +1 或者 -1
    """
    if p == q: 
        # 如果 p==q，那就是在同一个轨道上湮灭-产生，得到的组态是string0自己，这里不涉及费米子交换，符号为1
        return 1
    else:
        if (string0 & (1 << p)) or (not (string0 & (1 << q))):
            # 如果在p上本来有电子，或者q上没有电子，那么就无法产生/湮灭，这两个组态给出的矩阵元为0：
            return 0
        
        # 其他情况，则需要数一下产生算符p和湮灭算符q之间，交换了多少个电子，数一下p和q之间的1
        elif p > q:
            mask = (1 << p) - (1 << (q+1))
        else:
            mask = (1 << q) - (1 << (p+1))
        # 交换次数为偶数返回0， 为奇数返回1：
        return (-1) ** bin(string0 & mask).count('1')

def gen_linkstr_index_spin_orbital(orb_list, strings):
    """
    这个函数的作用是生成一个表，表格里记录了每个组态有哪些产生-湮灭的可能性。

    Args:
        orb_list (np.array): 所有自旋轨道的
        strings (np.array): _description_

    Returns:
        link_table: 
        返回一个长度为strings.shape[0]的np.array。
        link_table[n]中存储了第n个组态的所有产生湮灭可能。
    """
    # strdic = dict(zip(strings,range(strings.__len__())))
    def propagate1e(str0):
        occ = []
        vir = []
        for i in orb_list:
            if str0 & (1 << i):
                occ.append(i)
            else:
                vir.append(i)
        linktab = []
        for i in occ:
            # linktab.append((i, i, strdic[str0], 1))
            linktab.append((i, i, str0, 1))
        for i in occ:
            for a in vir:
                if i%2==a%2:
                    str1 = str0 ^ (1 << i) | (1 << a)
                    linktab.append((a, i, str1, cre_des_sign(a, i, str0)))
                else:
                    pass
        return linktab

    t = [propagate1e(s) for s in strings.astype(np.int64)]
    return t

In [5]:
link_table = gen_linkstr_index_spin_orbital(range(2*norb),ci_strings)
link_table[0][8]

(8, 0, 510, -1)

这个link_table是一个表格，它里面记录了每个组态所有可能的产生-湮灭算符组合。比如link_table[n]就表示了第n个组态的所有产生湮灭组合

每一项的四元素元组每个元素分别是(产生，湮灭，目标组态，符号)。

我们随便选一行进行说明，（8，0，510，-1）意思是对于0号组态（255，1111111100000000），在第8个轨道产生一个电子，在第0个轨道湮灭一个电子，会得到一个新组态(510,0111111110000000)，这个跃迁因为涉及奇数次费米子交换，所有需要乘以-1

之后同样是要计算哈密顿量矩阵元，这里和Hubbard Model的处理方式有点不一样。Hubbard Model的双电子项只需要考虑对角元，但是现在我们需要处理非对角元了。因为我们已经提前算了link_table，我们只需要对link_table遍历就可以了。

In [6]:
ham_mat = np.zeros((n_strings,n_strings))  # 初始化哈密顿量矩阵
str_dict = {s: idx for idx, s in enumerate(ci_strings)}

f1e = h1e - np.einsum('jiik->jk', eri) * .5  # 提前减掉这部分来避免double coutiing

# 处理单电子项
for idx0, tab in enumerate(link_table):  # 遍历 link_table
    for p, q, str1, sign in tab:         # 遍历所有产生湮灭算符组合
        idx1 = str_dict.get(str1)
        ham_mat[idx0,idx1] += sign* f1e[p//2,q//2]

# 处理双电子项
for idx0, tab in enumerate(link_table):
    for p, q, str1, sign1 in tab:
        idx1 = str_dict.get(str1)
        for r, s, str2, sign2 in link_table[idx1]:
            idx2 = str_dict.get(str2)
            ham_mat[idx0,idx2] += 0.5*sign1*sign2*eri[p//2,q//2,r//2,s//2]

遍历完link_table以后ham_mat就构建好了。之后把这个矩阵对角化。

In [7]:
import scipy.linalg
e,v =scipy.linalg.eigh(ham_mat)
fci_e = e[0] + mf.energy_nuc() # 别忘了加上核排斥能
fci_e

-4.307571602006763

验证一下和pyscf.fci计算结果是否相同

In [8]:
from pyscf.fci import direct_spin1
myfci = direct_spin1.FCI()
pyscf_fci_e,fci_v = myfci.kernel(h1e,eri,norb,nelec,ecore = mf.energy_nuc())
fci_e - pyscf_fci_e

-7.7964301681277e-12

可见是一致的。

FCI方法的问题是，随着norb和nelec增加，全空间里的组态数目是以阶乘增加的

In [9]:
get_num_configuration(20,20)

34134779536

20轨道20电子的体系就有1e11级别的组态数了，要构建 1e11 * 1e11的FCI矩阵并且对角化是不现实的。一个可行的方法是限制组态空间的规模，比如CISD方法就是只考虑相对HF态单双激发的组态，从而限制组态空间的大小并进行计算。

我们先确定在CISD计算中需要考虑的组态有哪些

In [10]:
import itertools

def generate_cisd(hf_config):
    occupied = [i for i, occ in enumerate(hf_config) if occ == 1]
    virtual = [i for i, occ in enumerate(hf_config) if occ == 0]
    cisd_configs = [hf_config.copy()]  # 包含原始HF组态
    # 生成单激发组态（Singly excited）
    for i in occupied:
        for a in virtual:
            if i%2 != a%2:  # 自旋检查
                continue
            new_config = hf_config.copy()
            new_config[i] = 0
            new_config[a] = 1
            cisd_configs.append(new_config)
    
    # 生成双激发组态（Doubly excited）
    for (i, j) in itertools.combinations(occupied, 2):
        for (a, b) in itertools.combinations(virtual, 2):
            if ((i%2==a%2) and (j%2==b%2)) or ((i%2==b%2) and (j%2==a%2)): #自旋检查
                new_config = hf_config.copy()
                new_config[i] = 0
                new_config[j] = 0
                new_config[a] = 1
                new_config[b] = 1
                cisd_configs.append(new_config)
    cisd_configs = np.array(cisd_configs)
    return cisd_configs
hf_state = np.zeros(2*norb)
hf_state[:nelec]=1
cisd_configs = generate_cisd(hf_state)
cisd_strings = config_to_string(cisd_configs)
n_cisd_strings = cisd_strings.shape[0]

构建CISD矩阵

In [11]:
ham_mat_cisd = np.zeros((n_cisd_strings,n_cisd_strings))  # 初始化哈密顿量矩阵
str_dict = {s: idx for idx, s in enumerate(ci_strings)}
str_dict_cisd = {s: idx for idx, s in enumerate(cisd_strings)}

f1e = h1e - np.einsum('jiik->jk', eri) * .5  # 提前减掉这部分来避免double coutiing

# 处理单电子项
for idx0, str0 in enumerate(cisd_strings):
    tab = link_table[str_dict.get(str0)]
    for p, q, str1, sign in tab:         
        if str1 in cisd_strings:
            idx1 = str_dict_cisd.get(str1)
            ham_mat_cisd[idx0,idx1] += sign* f1e[p//2,q//2]

# 处理双电子项
# for idx0, tab in enumerate(link_table):
for idx0, str0 in enumerate(cisd_strings):
    tab = link_table[str_dict.get(str0)]
    for p, q, str1, sign1 in tab:
        # if not (str1 in cisd_strings):
        #     continue
        idx1 = str_dict.get(str1)
        for r, s, str2, sign2 in link_table[idx1]:
            if str2 in cisd_strings:
                idx2 = str_dict_cisd.get(str2)
                ham_mat_cisd[idx0,idx2] += 0.5*sign1*sign2*eri[p//2,q//2,r//2,s//2]

然后对角化这个矩阵

In [12]:
import scipy.linalg
e,v =scipy.linalg.eigh(ham_mat_cisd)
cisd_e = e[0] + mf.energy_nuc() # 别忘了加上核排斥能
cisd_e

-4.297799976270009

使用PySCF的CISD也算出同样的结果

In [13]:
from pyscf import ci

mycisd = ci.RCISD(mf)
e,v = mycisd.kernel()
mycisd.e_tot - cisd_e

E(RCISD) = -4.297799976222395  E_corr = -0.1234301658332373


4.7614356901704014e-11

In [14]:
ci_strings.shape[0]

4900

In [15]:
cisd_strings.shape[0]

361

（上面我们写的FCI算法和CISD算法效率很低，不适合用于大体系计算，仅作算法正确性验证）